# Function Calling & Custom Agent Tools

Understanding differences and how to use them together

In [25]:
import pandas as pd
import numpy as np
import json, os, pprint
import streamlit as st
import matplotlib.pyplot as plt
import plotly.express as px
import random
from streamlit_jupyter import StreamlitPatcher
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.output_parsers import JsonOutputToolsParser, JsonOutputKeyToolsParser
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain import hub
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.callbacks import Callbacks
from langchain.prompts import ChatPromptTemplate

StreamlitPatcher().jupyter()

In [2]:
os.environ["OPENAI_API_KEY"] = ""

## Function Calling (Tool_Choice)

#### Converting a custom python function to an OpenAI function calling tool

Requires specifying type of arguments, as well as description of function and each arguments. The python function will be converted by convert_to_openai_tool(). More info [here](https://python.langchain.com/docs/modules/model_io/chat/function_calling)

**Note** Only functions are supported as tools by OpenAI. "tool_choice" controls if a tool should be used, and can force it to be used. More on this [here](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools)

*tools* and *tool_choice* replace the deprecated *functions* and *function_calling*, respectively

In [14]:
def create_matrix(r:int, c:int):
    """Create a matrix of rows(r) by columns(c)

    Args:
        r: First dimension of matrix
        c: Second dimension of matrix
    """

    m = np.random.randn(r, c)

    return m

convert_to_openai_tool(create_matrix)

{'type': 'function',
 'function': {'name': 'create_matrix',
  'description': 'Create a matrix of rows(r) by columns(c)',
  'parameters': {'type': 'object',
   'properties': {'r': {'type': 'integer',
     'description': 'First dimension of matrix'},
    'c': {'type': 'integer', 'description': 'Second dimension of matrix'}},
   'required': ['r', 'c']}}}

In [10]:

print(json.dumps(convert_to_openai_tool(create_matrix), indent=2))

{
  "type": "function",
  "function": {
    "name": "create_matrix",
    "description": "Create a matrix of rows(r) by columns(c)",
    "parameters": {
      "type": "object",
      "properties": {
        "r": {
          "type": "integer",
          "description": "First dimension of matrix"
        },
        "c": {
          "type": "integer",
          "description": "Second dimension of matrix"
        }
      },
      "required": [
        "r",
        "c"
      ]
    }
  }
}


### Testing ("Binding function) custom function call built in an LLM

We'll incorporate the function call as part of our model so it is "bound" (**binding function**) and ready to be utilized

In [3]:
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.1, streaming=True)

Test it without function calling. LLM will produce actual answer

In [136]:
llm_result = llm.invoke("can you create a random 3x5 matrix?")
llm_result

AIMessage(content="Sure, here's a random 3x5 matrix:\n\n\\[\n\\begin{pmatrix}\n7 & 3 & 5 & 2 & 8 \\\\\n1 & 4 & 9 & 6 & 0 \\\\\n2 & 7 & 8 & 3 & 4 \\\\\n\\end{pmatrix}\n\\]\n\nRemember, the numbers are randomly chosen, so they don't follow any specific pattern or rule.")

Notice that answer will be produced, but it will be produced as part of text

In [19]:
llm_result.content

'Sure, here is a random 3x5 matrix:\n\n[[4, 7, 2, 9, 1],\n [3, 8, 5, 6, 2],\n [1, 9, 4, 7, 3]]'

LLM with function calling will produce json input to pre-specified function ("create_matrix()" in function call)

In [21]:
llm_result_fc = llm.invoke("can you create a random 3x5 matrix?", 
                        tools = [convert_to_openai_tool(create_matrix)])

llm_result_fc

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hSJ0FnN67g7Itf0A3ZkLfyb8', 'function': {'arguments': '{"r":3,"c":5}', 'name': 'create_matrix'}, 'type': 'function'}]})

Notice that content will be null, but this will be report json input ready to be passed to actual function

In [23]:
llm_result_fc.content

''

In [64]:
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["name"])
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["arguments"])

{"r":3,"c":5}
create_pandas


**Binding permanently attaches the function to the LLM so that we can invoke it without calling the function call every time**

In [13]:
llm_fc = llm.bind(tools = [convert_to_openai_tool(create_matrix)])
llm_fc.invoke("can you create a random 3x5 matrix?") #Same answer as above

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EiUWczb5dViyT4Ls4m1g0Guk', 'function': {'arguments': '{"r":3,"c":5}', 'name': 'create_matrix'}, 'type': 'function'}]})

In the above example the regular llm was able to produce an answer for a simple request, but what would happen with a more complex request?

In [52]:
llm_result = llm.invoke("can you create a pandas dataframe of 3x5 with random numbers?")
llm_result

AIMessage(content="Sure! Here is an example code to create a pandas dataframe of size 3x5 with random numbers:\n\n```python\nimport pandas as pd\nimport numpy as np\n\ndata = np.random.randint(0, 100, size=(3, 5))\ndf = pd.DataFrame(data, columns=['A', 'B', 'C', 'D', 'E'])\n\nprint(df)\n```\n\nThis code will create a pandas dataframe with 3 rows and 5 columns filled with random numbers between 0 and 100.")

In [54]:
print(llm_result.content)

Sure! Here is an example code to create a pandas dataframe of size 3x5 with random numbers:

```python
import pandas as pd
import numpy as np

data = np.random.randint(0, 100, size=(3, 5))
df = pd.DataFrame(data, columns=['A', 'B', 'C', 'D', 'E'])

print(df)
```

This code will create a pandas dataframe with 3 rows and 5 columns filled with random numbers between 0 and 100.


In [58]:
def create_pandas(r:int, c:int):
    """Create a pandas dataframe of random numbers of rows(r) by columns(c)

    Args:
        r: First dimension of pandas dataframe
        c: Second dimension of pandas dataframe
    """

    import numpy as np
    import pandas as pd

    m  = np.random.randn(r, c)
    df = pd.DataFrame(m, columns=["a", "b", "c", "d", "e"])

    return df

print(json.dumps(convert_to_openai_tool(create_pandas), indent=2))

{
  "type": "function",
  "function": {
    "name": "create_pandas",
    "description": "Create a pandas dataframe of random numbers of rows(r) by columns(c)",
    "parameters": {
      "type": "object",
      "properties": {
        "r": {
          "type": "integer",
          "description": "First dimension of pandas dataframe"
        },
        "c": {
          "type": "integer",
          "description": "Second dimension of pandas dataframe"
        }
      },
      "required": [
        "r",
        "c"
      ]
    }
  }
}


In [59]:
llm_result_fc = llm.invoke("can you create a pandas dataframe of 3x5 with random numbers?", 
                        tools = [convert_to_openai_tool(create_pandas)])

llm_result_fc

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ecz0zRIuBdKKSunS9rPGAcsI', 'function': {'arguments': '{"r":3,"c":5}', 'name': 'create_pandas'}, 'type': 'function'}]})

In [65]:
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["name"])
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["arguments"])

create_pandas
{"r":3,"c":5}


OK....how about something much more complex, like a figure?

In [66]:
llm_result = llm.invoke("can you create a streamlit linechart showing 2 lines with random numbers going from 0 to 100?")
llm_result

AIMessage(content="Sure! Here is an example code to create a Streamlit line chart with 2 lines showing random numbers going from 0 to 100:\n\n```python\nimport streamlit as st\nimport pandas as pd\nimport numpy as np\n\n# Generate random data\ndata = pd.DataFrame({\n    'x': np.arange(0, 101),\n    'y1': np.random.randint(0, 101, size=101),\n    'y2': np.random.randint(0, 101, size=101)\n})\n\n# Create line chart\nst.line_chart(data[['y1', 'y2']])\n```\n\nYou can run this code in a Python environment with Streamlit installed to see the line chart with 2 lines showing random numbers.")

In [68]:
print(llm_result.content)

Sure! Here is an example code to create a Streamlit line chart with 2 lines showing random numbers going from 0 to 100:

```python
import streamlit as st
import pandas as pd
import numpy as np

# Generate random data
data = pd.DataFrame({
    'x': np.arange(0, 101),
    'y1': np.random.randint(0, 101, size=101),
    'y2': np.random.randint(0, 101, size=101)
})

# Create line chart
st.line_chart(data[['y1', 'y2']])
```

You can run this code in a Python environment with Streamlit installed to see the line chart with 2 lines showing random numbers.


In [8]:
def create_streamlit_lines(n:int, d:int, l_range:int, h_range:int):
    """Create a streamlit linechart of n number of lines
    of a random distribution of d numbers with the lowest number 
    being higher or equal to l_range
    and the highest number being lower or equal to h_range

    Args:
        n: number of lines
        d: sample size of distribution
        l_range: lowest number of lines random distribution
        h_range: highest number of lines random distribution

    """

    import numpy as np
    import pandas as pd
    import streamlit as st

    df = pd.DataFrame({
        'x': np.arange(0, d),
        'y1': np.random.randint(l_range, h_range, size=d),
        'y2': np.random.randint(l_range, h_range, size=d)
    })

    plot = st.line_chart(df[['y1', 'y2']])

    return(plot)

print(json.dumps(convert_to_openai_tool(create_streamlit_lines), indent=2))

{
  "type": "function",
  "function": {
    "name": "create_streamlit_lines",
    "description": "Create a streamlit linechart of n number of lines\nof a random distribution of d numbers with the lowest number \nbeing higher or equal to l_range\nand the highest number being lower or equal to h_range",
    "parameters": {
      "type": "object",
      "properties": {
        "n": {
          "type": "integer",
          "description": "number of lines"
        },
        "d": {
          "type": "integer",
          "description": "sample size of distribution"
        },
        "l_range": {
          "type": "integer",
          "description": "lowest number of lines random distribution"
        },
        "h_range": {
          "type": "integer",
          "description": "highest number of lines random distribution"
        }
      },
      "required": [
        "n",
        "d",
        "l_range",
        "h_range"
      ]
    }
  }
}


In [9]:
llm_result_fc = llm.invoke("Create a streamlit figure containing 2 lines, each line being of size 10", 
                        tools = [convert_to_openai_tool(create_streamlit_lines)])

llm_result_fc

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BTT5GxtClxWgi3msHs8vSoo3', 'function': {'arguments': '{"n": 2, "d": 10, "l_range": 1, "h_range": 10}', 'name': 'create_streamlit_lines'}, 'type': 'function'}, {'id': 'call_wAHV1JZIrJiNzg8CD31wbZY3', 'function': {'arguments': '{"n": 2, "d": 10, "l_range": 1, "h_range": 10}', 'name': 'create_streamlit_lines'}, 'type': 'function'}]})

In [10]:
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["name"])
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["arguments"])

create_streamlit_lines
{"n": 2, "d": 10, "l_range": 1, "h_range": 10}


As seen the function caller does a pretty good job at it, in numpy, pandas and more complex streamlit function calls. **Even when not having all arguments provided by the user, it makes pretty decent assumptions**

Now that we know how to function call, we'll dive next into actually using the tools, and passing the arguments to the tools that the LLM function calls

## Tool Use

Inspiration from [here](https://python.langchain.com/docs/use_cases/tool_use/)

There are **two ways** to interface(*invoke*) with tools.

1. With a *chain* for pre-defined sequence of tool usage:

<p>
<img src="ILLUSTRATIONS/tool_chain_sequence_diagram.svg" 
      width="65%" height="auto"
      style="display: block; margin: 0 auto" />

2. With an *agent loop* so the agent decides how many times to use the tool:

<p>
<img src="ILLUSTRATIONS/tool_agent_loop_diagram.svg" 
      width="65%" height="auto"
      style="display: block; margin: 0 auto" />



### 1. Let's explore the first one, **Tool Chain** for pre-defined usage of a tool. 

We'll create a custom tool first using the *tool decorator* on the plotly express dataset

<p>
<img src="ILLUSTRATIONS/tool_chain_sequence_diagram.svg" 
      width="45%" height="auto"
      style="display: block; margin: 0 auto" />

We want to develop a tool which can create histograms of pre-defined columns. We'll use the plotly express as an example. For reference the native OpenAI llm does not produce an accurate response...

In [43]:
llm.invoke("Create a linechart with the stocks dataframe from plotly express comparing the stock price over time of GOOG and APPL with a different color per stock")

AIMessage(content="import plotly.express as px\n\nfig = px.line(stocks, x='Date', y='Close', color='Stock')\nfig.show()")

In [4]:
@tool
def px_trend_compare(data:str, col1:str, col2:str):
    """Creates a line chart using plotly of two columns
    'col1' and 'col2,' which are two stock market ticker symbols,
    tracking their value over time. The data is
    available in 'data' stored in plotly express.
    Assign a different color for each line 'col1' and 'col2.'

    Args:
        data: Table name in plotly express
        col1: Stock market ticker symbol
        col2: Another stock market ticker symbol

    """

    import plotly.express as px

    data_module = getattr(px.data, data)
    df = data_module()

    df = (df
          .melt(id_vars="date", value_vars=[col1, col2])
    )

    fig = px.line(df, x='date', y='value', color='variable')
    
    return fig.show()

In [104]:
px.data.stocks()

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...,...
100,2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
101,2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
102,2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896
103,2019-12-23,1.226504,1.656000,1.521226,1.113728,1.567170,1.802472


This is now a **custom tool we can call in Langchain** with its own properties

In [45]:
print(px_trend_compare.name)
print(px_trend_compare.description)
print(px_trend_compare.args)

px_trend_compare
px_trend_compare(data: str, col1: str, col2: str) - Creates a line chart using plotly of two columns
    'col1' and 'col2,' which are two stock market ticker symbols,
    tracking their value over time. The data is
    available in 'data' stored in plotly express.
    Assign a different color for each line 'col1' and 'col2.'

    Args:
        data: Table name in plotly express
        col1: Stock market ticker symbol
        col2: Another stock market ticker symbol
{'data': {'title': 'Data', 'type': 'string'}, 'col1': {'title': 'Col1', 'type': 'string'}, 'col2': {'title': 'Col2', 'type': 'string'}}


Let's test out our plotting tool

In [48]:
px_trend_compare.invoke(
    {"data":"stocks", "col1":"GOOG", "col2":"AMZN"}
    )

Works like a charm!! Let's now use **function calling** to **bind** and convert our Langchain tool to a function call. Binding it to our LLM will make it callable all the time. *We can also enfore it by passing the argument 'tool_choice'*

In [49]:
llm_px_stocks = llm.bind_tools([px_trend_compare], tool_choice="px_trend_compare")

Now it is an LLM-bound function call

In [50]:
llm_px_stocks.kwargs["tools"]

[{'type': 'function',
  'function': {'name': 'px_trend_compare',
   'description': "px_trend_compare(data: str, col1: str, col2: str) - Creates a line chart using plotly of two columns\n    'col1' and 'col2,' which are two stock market ticker symbols,\n    tracking their value over time. The data is\n    available in 'data' stored in plotly express.\n    Assign a different color for each line 'col1' and 'col2.'\n\n    Args:\n        data: Table name in plotly express\n        col1: Stock market ticker symbol\n        col2: Another stock market ticker symbol",
   'parameters': {'type': 'object',
    'properties': {'data': {'type': 'string'},
     'col1': {'type': 'string'},
     'col2': {'type': 'string'}},
    'required': ['data', 'col1', 'col2']}}}]

In [31]:
# print(json.dumps(convert_to_openai_tool(create_matrix), indent=2))
# print(json.dumps(convert_to_openai_tool(px_trend_compare), indent=2))

In [12]:
llm_px_stocks.kwargs["tool_choice"]

{'type': 'function', 'function': {'name': 'px_trend_compare'}}

Function calling works well!

In [13]:
llm_px_stocks.invoke("can you use the stocks dataframe in plotly express to generate a line graph comparing the stocks GOOG and AMZN?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bdNgkcUxgS49flHb8bdD9wfQ', 'function': {'arguments': '{"data":"stocks","col1":"GOOG","col2":"AMZN"}', 'name': 'px_trend_compare'}, 'type': 'function'}]})

We'll simply this response and add a JSON output parsers to convert this to a simple list. We'll start this chain

In [14]:
llm_chain = llm_px_stocks | JsonOutputToolsParser()

In [15]:
llm_chain.invoke("can you use the stocks dataframe in plotly express to generate a line graph comparing the stocks GOOG and AMZN?")

[{'type': 'px_trend_compare',
  'args': {'data': 'stocks', 'col1': 'GOOG', 'col2': 'AMZN'}}]

**NOTE:** If we know for certain we'll be **invoking** our px_trend_compare tool to answer this function call, we can specify to only return its arguments in our **JsonOutputKeyToolParser** and request to only retun the first element of a potential list of many tool invocations

In [16]:
llm_chain = llm_px_stocks | JsonOutputKeyToolsParser(
    key_name="px_trend_compare", return_single=True
)

llm_chain.invoke("can you use the stocks dataframe in plotly express to generate a line graph comparing the stocks GOOG and AMZN?")

{'data': 'stocks', 'col1': 'GOOG', 'col2': 'AMZN'}

Great! Now that we have the specific function call return in a simple (digestable) format (get tool invocation), we can finally **call the tool** to be executed

In [51]:
llm_chain = (
    llm_px_stocks
    | JsonOutputKeyToolsParser(
        key_name="px_trend_compare", 
        return_single=True)
    | px_trend_compare 
)

llm_chain.invoke("can you use the stocks dataframe in plotly express to generate a line graph comparing the stocks GOOG and AMZN?")

**We were able to bind a tool to an LLM, and invoke the tool to get a graphical output from ChatOpenAI** Let's try the Agent path now

### 2. Let's explore the second option, **Agents** letting an agent decide how many times and when to use a tool 

We'll build a couple of tools for this and use the OpenAI tool agent which is compatible with the OpenAI tool calling API. **Agents allow multi-tool use.**

<p>
<img src="ILLUSTRATIONS/tool_agent_loop_diagram.svg" 
      width="45%" height="auto"
      style="display: block; margin: 0 auto" />

Let's use a generic prompt from the langchain hub

In [19]:
prompt = hub.pull("hwchase17/openai-tools-agent")
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_

In [24]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


Now let's build our tools. We'll repurpose our *px_trend_compare* tool above.

In [43]:
tools = [px_trend_compare]
# tools = []

We'll construct the Tools agent and the Agent executor

In [42]:
agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [54]:
agent_executor.invoke({
    "input": "can you use the stocks dataframe in plotly express to generate a line graph with comparing the Google and Amazon stocks?"
})



> Entering new AgentExecutor chain...

Invoking: `px_trend_compare` with `{'col1': 'GOOG', 'col2': 'AMZN', 'data': 'stocks'}`




NoneI attempted to generate a line graph comparing Google and Amazon stocks using the stocks dataframe in Plotly Express, but it seems there was an issue, and I couldn't produce the graph. If there's anything else I can help you with or another way I can assist you, please let me know!

> Finished chain.


{'input': 'can you use the stocks dataframe in plotly express to generate a line graph with comparing the Google and Amazon stocks?',
 'output': "I attempted to generate a line graph comparing Google and Amazon stocks using the stocks dataframe in Plotly Express, but it seems there was an issue, and I couldn't produce the graph. If there's anything else I can help you with or another way I can assist you, please let me know!"}

It seems that it is using the correct Tool (which is great!), but not exactly returning a plot (which is what we want). Let's try to fix that by specifying the struture of our desired response

In [21]:
agent_executor.invoke({
    "input": "can you use the stocks dataframe in plotly express to generate a line graph comparing two stocks?"
})



> Entering new AgentExecutor chain...
Sure, I can help with that. Could you please specify which two stocks you would like to compare?

> Finished chain.


{'input': 'can you use the stocks dataframe in plotly express to generate a line graph comparing two stocks?',
 'output': 'Sure, I can help with that. Could you please specify which two stocks you would like to compare?'}

In [23]:
agent_executor({
    "input":"Apple is one of them"
})



> Entering new AgentExecutor chain...
It seems like you're referring to something specific about Apple, but I need a bit more context to provide a helpful response. Could you please provide more details or clarify your request?

> Finished chain.


{'input': 'Apple is one of them',
 'output': "It seems like you're referring to something specific about Apple, but I need a bit more context to provide a helpful response. Could you please provide more details or clarify your request?"}

In the example above the agent had no recollection of the prior conversation. Let's add **memory** to it. We'll **build the agent using *OpenAIToolsAgentOutputParser()*** and incorporate the **chat history**

**Notice** how we manually bind the tools to the llm

In [136]:
# We had this before: agent = create_openai_tools_agent(llm, tools, prompt)

# Now:
tools = [px_trend_compare]
llm_bound_tools = llm.bind_tools(tools)
chat_history = [] 

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]), # Part of the prompt as well
        "chat_history": lambda x: x["chat_history"] # Part of the prompt as well
    }
    | prompt
    | llm_bound_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Great, now let's test its execution and check if it remembers

In [137]:
input1 = "can you use the stocks dataframe in plotly express to generate a line graph comparing two stocks?"

ai_interaction = agent_executor.invoke({
    "input": input1,
    "chat_history" : chat_history #Integrating chat_history
})



> Entering new AgentExecutor chain...
Sure, I can help with that. Could you please specify which two stocks you would like to compare?

> Finished chain.


In [138]:
ai_interaction

{'input': 'can you use the stocks dataframe in plotly express to generate a line graph comparing two stocks?',
 'chat_history': [],
 'output': 'Sure, I can help with that. Could you please specify which two stocks you would like to compare?'}

We make sure to extend chat history manually and add it to the agent invocation

In [48]:
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content = ai_interaction["output"])
    ]
)

input2 = "Apple is one of them"

ai_interaction = agent_executor.invoke({
    "input": input2,
    "chat_history" : chat_history #Extending with previous integrating chat_history
})



> Entering new AgentExecutor chain...
Great choice! Could you please specify the second stock you'd like to compare with Apple?

> Finished chain.


In [49]:
ai_interaction

{'input': 'Apple is one of them',
 'chat_history': [HumanMessage(content='can you use the stocks dataframe in plotly express to generate a line graph comparing two stocks?'),
  AIMessage(content='Sure, I can help with that. Could you please specify which two stocks you would like to compare?')],
 'output': "Great choice! Could you please specify the second stock you'd like to compare with Apple?"}

**Memory was added!!**

### Implement Agent Streaming

Needed for App (e.g. StreamLit) interaction

We need to add **with_config() tags** to the agent's llm and agentexecutor to be referenced in the ```astream_events``` API. Reference [here](https://python.langchain.com/docs/modules/agents/how_to/streaming)

In [139]:
tools = [px_trend_compare]
llm_bound_tools = llm.bind_tools(tools)
chat_history = [] 

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]), # Part of the prompt as well
        "chat_history": lambda x: x["chat_history"] # Part of the prompt as well
    }
    | prompt
    | llm_bound_tools.with_config({"tags": ["agent_llm"]}) # THIS IS MODIFIED
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_config({"run_name":"Agent"}) #THIS IS MODIFIED

**The .astream method** outputs are action-observation pairs until an answer is achieved by the agent. The contents of the outputs are:
- *Actions*: the actions the agent will take: agentaction() or messages() referring to the agent invocation
- *Observations*: steps() with history of what agent has done so far and messages() with function invocation results.
- *Final answer*: Agentfinish() output and messages()

In [140]:
chunks = []
input1 = "can you use the stocks dataframe in plotly express to generate a line graph comparing amazon and apple?"

async for chunk in agent_executor.astream({
# for chunk in agent_executor.stream({
    "input": input1,
    "chat_history" : chat_history #Integrating chat_history
}):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)



> Entering new AgentExecutor chain...
------
{'actions': [...], 'messages': [...]}

Invoking: `px_trend_compare` with `{'col1': 'AAPL', 'col2': 'AMZN', 'data': 'stocks'}`




None------
{'messages': [...], 'steps': [...]}
I attempted to generate a line graph comparing Amazon and Apple using the stocks data from Plotly Express, but it seems there was an issue with the process. Unfortunately, I'm unable to provide the graph at this moment. Is there anything else I can assist you with?

> Finished chain.
------
{'messages': [...],
 'output': 'I attempted to generate a line graph comparing Amazon and Apple '
           'using the stocks data from Plotly Express, but it seems there was '
           "an issue with the process. Unfortunately, I'm unable to provide "
           'the graph at this moment. Is there anything else I can assist you '
           'with?'}


We can see the actions, observations and final outputs in the chunks

In [141]:
chunks

[{'actions': [OpenAIToolAgentAction(tool='px_trend_compare', tool_input={'col1': 'AAPL', 'col2': 'AMZN', 'data': 'stocks'}, log="\nInvoking: `px_trend_compare` with `{'col1': 'AAPL', 'col2': 'AMZN', 'data': 'stocks'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_61YIQWaUQDeu5vuAolFQ2hzu', 'function': {'arguments': '{"col1":"AAPL","col2":"AMZN","data":"stocks"}', 'name': 'px_trend_compare'}, 'type': 'function'}]})], tool_call_id='call_61YIQWaUQDeu5vuAolFQ2hzu')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_61YIQWaUQDeu5vuAolFQ2hzu', 'function': {'arguments': '{"col1":"AAPL","col2":"AMZN","data":"stocks"}', 'name': 'px_trend_compare'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='px_trend_compare', tool_input={'col1': 'AAPL', 'col2': 'AMZN', 'data': 'stocks'}, log="\nInvoking: `px_trend_compare` with `{'col1': 'AAPL', 'col2': 'AMZN',

In [142]:
for i in chunks:
    print(i["messages"])

[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_61YIQWaUQDeu5vuAolFQ2hzu', 'function': {'arguments': '{"col1":"AAPL","col2":"AMZN","data":"stocks"}', 'name': 'px_trend_compare'}, 'type': 'function'}]})]
[FunctionMessage(content='null', name='px_trend_compare')]
[AIMessage(content="I attempted to generate a line graph comparing Amazon and Apple using the stocks data from Plotly Express, but it seems there was an issue with the process. Unfortunately, I'm unable to provide the graph at this moment. Is there anything else I can assist you with?")]


We can format the output (during streaming) to give us an idea of how to display it in our app:

In [68]:
for chunk in chunks:
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(step, "\n")
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")

Calling Tool: `px_trend_compare` with input `{'col1': 'AAPL', 'col2': 'AMZN', 'data': 'stocks'}`
---
action=OpenAIToolAgentAction(tool='px_trend_compare', tool_input={'col1': 'AAPL', 'col2': 'AMZN', 'data': 'stocks'}, log="\nInvoking: `px_trend_compare` with `{'col1': 'AAPL', 'col2': 'AMZN', 'data': 'stocks'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_BFkyWbb9vvpUFCXoiRmUzDuS', 'function': {'arguments': '{"col1":"AAPL","col2":"AMZN","data":"stocks"}', 'name': 'px_trend_compare'}, 'type': 'function'}]})], tool_call_id='call_BFkyWbb9vvpUFCXoiRmUzDuS') 

Tool Result: `None`
---
Final Output: I attempted to generate a line graph comparing Amazon and Apple using the stocks dataframe in Plotly Express, but it seems there was an issue with the process. Unfortunately, I'm unable to display the graph at this moment. Is there anything else I can assist you with?
---


In [99]:
agent_stream_response = agent_executor.stream({
    "input": input1,
    "chat_history" : chat_history #Integrating chat_history
})

In [101]:
agent_stream_response

<generator object RunnableBindingBase.stream at 0x157291ba0>

In [100]:
st.write_stream(agent_stream_response)



> Entering new AgentExecutor chain...


2024-03-08 00:30:17.891 
  command:

    streamlit run /opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]



Invoking: `px_trend_compare` with `{'col1': 'AMZN', 'col2': 'AAPL', 'data': 'stocks'}`




None
Invoking: `px_trend_compare` with `{'data': 'stocks', 'col1': 'AMZN', 'col2': 'AAPL'}`
responded: I've generated the comparison, but it seems there was an issue displaying the graph. Let me try again to ensure you get the visual comparison between Amazon (AMZN) and Apple (AAPL) stock prices.



NoneIt appears there's a persistent issue preventing the display of the line graph comparing Amazon (AMZN) and Apple (AAPL) stock prices. Unfortunately, I'm unable to provide the visual comparison at this moment. If you have any other requests or need assistance with a different query, please let me know!

> Finished chain.


[{'actions': [OpenAIToolAgentAction(tool='px_trend_compare', tool_input={'col1': 'AMZN', 'col2': 'AAPL', 'data': 'stocks'}, log="\nInvoking: `px_trend_compare` with `{'col1': 'AMZN', 'col2': 'AAPL', 'data': 'stocks'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_jmkCDayV5A5JNtGbABBZdQjL', 'function': {'arguments': '{"col1":"AMZN","col2":"AAPL","data":"stocks"}', 'name': 'px_trend_compare'}, 'type': 'function'}]})], tool_call_id='call_jmkCDayV5A5JNtGbABBZdQjL')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_jmkCDayV5A5JNtGbABBZdQjL', 'function': {'arguments': '{"col1":"AMZN","col2":"AAPL","data":"stocks"}', 'name': 'px_trend_compare'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='px_trend_compare', tool_input={'col1': 'AMZN', 'col2': 'AAPL', 'data': 'stocks'}, log="\nInvoking: `px_trend_compare` with `{'col1': 'AMZN', 'col2': 'AAPL',

### Evaluate additional prompt types suitable for Apps

**OpenAI Tool Agent Prompt**. Has *messages*

In [5]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [6]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [89]:
from langchain.prompts import SystemMessagePromptTemplate, PromptTemplate


system_prompt ="""
You are a helpful assistant. Assistant has access to the following tools: {tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [one of the tools provided]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

"""

prompt.messages[0] = (
    SystemMessagePromptTemplate(prompt = PromptTemplate(input_variables=["tools"],
                                                        template=system_prompt))
)

prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tools'], template='\nYou are a helpful assistant. Assistant has access to the following tools: {tools}\n\nTo use a tool, please use the following format:\n\n```\nThought: Do I need to use a tool? Yes\nAction: the action to take,

In [92]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tools'], template='\nYou are a helpful assistant. Assistant has access to the following tools: {tools}\n\nTo use a tool, please use the following format:\n\n```\nThought: Do I need to use a tool? Yes\nAction: the action to take, should be one of [one of the tools provided]\nAction Input: the input to the action\nObservation: the result of the action\n```\n\nWhen you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:\n\n```\nThought: Do I need to use a tool? No\nFinal Answer: [your response here]\n```\n\n')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [93]:
prompt.pretty_print()

================================ System Message ================================


You are a helpful assistant. Assistant has access to the following tools: {tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [one of the tools provided]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```



============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


**Reach Prompt**. No messages

In [8]:
react_prompt = hub.pull("hwchase17/react")
react_prompt.messages

AttributeError: 'PromptTemplate' object has no attribute 'messages'

In [9]:
react_prompt.pretty_print()

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


**React + Chat**. No *messages*

In [10]:
react_chat_prompt = hub.pull("hwchase17/react-chat")
react_chat_prompt

PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descr

In [12]:
react_chat_prompt.messages

AttributeError: 'PromptTemplate' object has no attribute 'messages'

In [13]:
react_chat_prompt.pretty_print()

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

Let's test these with a dummy example

In [21]:
@tool
def where_cat_is_hiding() -> str:
    """Where is the cat hiding right now?"""
    return random.choice(["under the bed", "on the shelf"])


@tool
def get_items(place: str) -> str:
    """Use this tool to look up which items are in the given place."""
    if "bed" in place:  # For under the bed
        return "socks, shoes and dust bunnies"
    if "shelf" in place:  # For 'shelf'
        return "books, penciles and pictures"
    else:  # if the agent decides to ask about a different place
        return "cat snacks"

 #### Testing OpenAI Tool Agent Prompt

In [100]:
tools = [where_cat_is_hiding, get_items]
llm_bound_tools = llm.bind_tools(tools)
chat_history = [] 

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]), # Part of the prompt as well
        "chat_history": lambda x: x["chat_history"], # Part of the prompt as well,
        "tools": lambda x: " and ".join([i.get_name() for i in x["tools"]]) 
    }
    | prompt
    | llm_bound_tools.with_config({"tags": ["agent_llm"]}) # THIS IS MODIFIED
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_config({"run_name":"Agent"}) #THIS IS MODIFIED

In [102]:
from langchain_community.callbacks.streamlit import StreamlitCallbackHandler

st_callback = StreamlitCallbackHandler(st.container())

agent_executor_openai_prompt_output = agent_executor.invoke({
    "input": "what's items are located where the cat is hiding?",
    "chat_history": chat_history,
    "tools":tools
    },
    {
    "callbacks": [st_callback]
    }
    )



> Entering new AgentExecutor chain...


Error in StreamlitCallbackHandler.on_tool_start callback: StreamlitAPIException()



Invoking: `where_cat_is_hiding` with `{}`




Error in StreamlitCallbackHandler.on_tool_end callback: StreamlitAPIException()


on the shelf

...


Invoking: `get_items` with `{'place': 'on the shelf'}`




Error in StreamlitCallbackHandler.on_tool_end callback: StreamlitAPIException()


books, penciles and pictures

Error in StreamlitCallbackHandler.on_agent_finish callback: StreamlitAPIException()


Final Answer: The items located where the cat is hiding, on the shelf, are books, pencils, and pictures.

> Finished chain.


#### Testing React Agent Prompt

Notice that it has *no chat_history*

In [24]:
react_prompt.pretty_print()

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [112]:
react_agent = create_react_agent(llm, tools, react_prompt)
# react_agent

Compared to the *OpenAI Tools Agent* prompt, the react agent prompt has *no* `ChatpromptTemplate` and the output is `ReActSingleInputOutputParser`

In [111]:
# agent

In [37]:
react_agent_executor = AgentExecutor(agent=react_agent, tools=tools, verbose=True) #.with_config({"run_name":"Agent"}) 

agent_executor_react_prompt_output = react_agent_executor.invoke({
    "input": "what's items are located where the cat is hiding?",
    # "chat_history": chat_history
})



> Entering new AgentExecutor chain...
First, I need to find out where the cat is hiding.
Action: where_cat_is_hiding
Action Input: None

RuntimeError: generator raised StopIteration

**React prompt agent won't work with Chat models, only OpenAI non-chat model**. Let's see if this changes in the React Chat Agent prompt

#### Testing React Chat Agent Prompt

Notice that this one has *chat_history*

In [38]:
react_chat_prompt.pretty_print()

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [83]:
tools = [where_cat_is_hiding, get_items]
llm_bound_tools = llm.bind_tools(tools)
chat_history = [] 

react_chat_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]), # Part of the prompt as well
        "chat_history": lambda x: x["chat_history"], # Part of the prompt as well
        "tools": tools,
        "tool_names": "\n".join([i.get_name() for i in tools])
    }
    | react_chat_prompt
    | llm_bound_tools.with_config({"tags": ["agent_llm"]}) # THIS IS MODIFIED
    | OpenAIToolsAgentOutputParser()
)

react_chat_agent_executor = AgentExecutor(agent=react_chat_agent, tools=tools, verbose=True).with_config({"run_name":"Agent"}) #THIS IS MODIFIED

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>

The react chat agent has chat messaging

In [57]:
# react_chat_agent

In [80]:
agent_executor_react_chat_prompt_output = react_chat_agent_executor.invoke({
    "input": "what's items are located where the cat is hiding?",
    "chat_history": chat_history,
})



> Entering new AgentExecutor chain...


KeyError: "Input to PromptTemplate is missing variables {'tools', 'tool_names'}.  Expected: ['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] Received: ['agent_scratchpad', 'input', 'chat_history']"

In [54]:
agent_executor_react_chat_prompt_output

{'input': "what's items are located where the cat is hiding?",
 'chat_history': [],
 'output': 'The items located where the cat is hiding, under the bed, are socks, shoes, and dust bunnies.'}